In [1]:
import pickle
import random
import numpy as np
import pandas as pd

with open('../tokens', 'rb') as pickle_file:
    tokenized_sent = pickle.load(pickle_file)

random.shuffle(tokenized_sent)

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

train_tokens = [' '.join(token[2]) for token in tokenized_sent]
themes = [token[1] for token in tokenized_sent]

In [2]:
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [4]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [5]:
sentence_embeddings = model(train_tokens)

In [6]:
tokens_embedding = model(train_tokens) 

In [7]:
end_tokens = False

gap = 5000
offset = 0
limit = gap
length = len(tokens_embedding)


first_loop = True

In [8]:
while not end_tokens:

    if limit > length:
        limit = length
        end_tokens = True
    
    sub_tokens_embedding = tokens_embedding[offset:limit]
    results = []
    for index, token in enumerate(sub_tokens_embedding):
        row = np.append(token, themes[index + offset])
        results.append(row)

    df = pd.DataFrame.from_dict(results)

    offset += gap
    limit += gap
    
    df.to_csv('../dataframes/universal_sentence_python.csv', index=False, encoding='utf-8', mode='a', header=False)